<a href="https://colab.research.google.com/github/hdilab/hpm/blob/master/Char-LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [43]:
import numpy as np
import torch
from torch import nn
import torch.nn.functional as F
import os
Colab = False
NumOnBits = 10
NumBits = 512
Seed = 42

In [44]:
if Colab:
    from google.colab import drive
    drive.mount('/content/drive')
    with open('/content/drive/My Drive/Colab/data/short.txt','r') as f:
        text = f.read()
else:
    with open('data/1342.txt','r') as f:
        text = f.read()

In [45]:
text[:100]

'The Project Gutenberg EBook of Pride and Prejudice, by Jane Austen\n\nThis eBook is for the use of any'

In [46]:
asc_chars = [chr(i) for i in range(128)]
chars = tuple(asc_chars)
int2char = dict(enumerate(chars))
char2int = {c:i for i, c in int2char.items()}

encoded = np.array([char2int[ch] for ch in text])
encoded[:100]

array([ 84, 104, 101,  32,  80, 114, 111, 106, 101,  99, 116,  32,  71,
       117, 116, 101, 110,  98, 101, 114, 103,  32,  69,  66, 111, 111,
       107,  32, 111, 102,  32,  80, 114, 105, 100, 101,  32,  97, 110,
       100,  32,  80, 114, 101, 106, 117, 100, 105,  99, 101,  44,  32,
        98, 121,  32,  74,  97, 110, 101,  32,  65, 117, 115, 116, 101,
       110,  10,  10,  84, 104, 105, 115,  32, 101,  66, 111, 111, 107,
        32, 105, 115,  32, 102, 111, 114,  32, 116, 104, 101,  32, 117,
       115, 101,  32, 111, 102,  32,  97, 110, 121])

In [47]:
"""
SDR class
Handles issues with SDR
Given a char input, generate SDR
"""

import random


class SDR(object):
    """
      Class implementing the SDR.

      :param input_list: (List) List for input_values.
            For ASCII it will be [chr(0), chr(1), ... chr(127)]

      :param numBits: (int) Number of bits for SDR. Default value ``512``

      :param numOnBits: (int) Number of Active bits for SDR. Default value ``10``.
            It is 2% sparcity for 512 bit

      :param seed: (int) Seed for the random number generator. Default value ``42``.
    """

    def __init__(self,
                 input_list,
                 numBits=512,
                 numOnBits=10,
                 seed=42,
                 inputNoise=0.1):

        random.seed(seed)
        self.population = [i for i in range(numBits)]
        self.numOnBits = numOnBits
        self.inputNoise = inputNoise
        self.sdr_dict = {i:random.sample(self.population, numOnBits) for i in input_list}


    def getSDR(self, input):
        return self.sdr_dict[input]


    def getNoisySDR(self, input):
        inputSDR = self.sdr_dict[input]
        inputSDR = [i for i in inputSDR if random.random() > self.inputNoise]
        noise = random.sample(self.population, int(self.numOnBits * self.inputNoise))
        return inputSDR + noise



    def getInput(self, sdr):
        """
        Need to implement the function which returns the corresponding input from SDR
        This requires a probabilistic approach. Count the number of overlapping bit and nonoverlapping field.
        """
        return 0

    def getCollisionProb(self, n, a, s, theta):
        """
        Calculating the probability for the cases where more than theta synapses are activated
        for different cell activation pattern
        :param n: Number of cells
        :param a: Number of active cells
        :param s: Number of synapses
        :param theta: Threshold for the dendritic activation
        :return: The probability where dendritic activation for the different cell activation pattern
        """
        numerator = 0
        for b in range(theta, s+1):
            numerator += combinatorial(s, b) * combinatorial(n-s, a-b)

        denominator = combinatorial(n, a)

        return numerator*1.0/denominator

    def getRandomSDR(self):
        noise = random.sample(self.population, numOnBits)
        return noise


def combinatorial(a,b):
    return factorial(a)*1.0/factorial(a-b)/factorial(a)

def factorial(a):
    if a == 1:
        return 1
    else:
        return a*factorial(a-1)



In [48]:
char_sdr = SDR(asc_chars,
                numBits=NumBits,
                numOnBits=NumOnBits,
                seed=Seed,
                inputNoise=0.1)

In [49]:
def one_hot_encoder(arr, n_labels):
    one_hot = np.zeros((np.multiply(*arr.shape), n_labels), dtype=np.float32)
    one_hot[np.arange(one_hot.shape[0]), arr.flatten()] = 1. 
    one_hot = one_hot.reshape((*arr.shape, n_labels))
    return one_hot

def multi_hot_encoder(arr, n_labels):
    multi_hot = np.zeros((arr.shape[0], arr.shape[1], n_labels), dtype=np.float32)
    for i in range(arr.shape[0]):
        for j in range(arr.shape[1]):
            sdr = char_sdr.getNoisySDR(int2char[arr[i][j]])
            multi_hot[i][j][np.array(sdr)] = 1  
    return multi_hot

In [50]:
test_seq = np.array([[3,5,1]])
one_hot=one_hot_encoder(test_seq, 8)
multi_hot = multi_hot_encoder(test_seq, 512)

In [51]:
print (test_seq.shape)

(1, 3)


In [52]:
print(one_hot)

[[[0. 0. 0. 1. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 1. 0. 0.]
  [0. 1. 0. 0. 0. 0. 0. 0.]]]


In [53]:
print(multi_hot.shape)

(1, 3, 512)


In [54]:
sdr = char_sdr.getNoisySDR(int2char[1])
a = np.zeros((3,512))
a[1][np.array(sdr)] = 1
print (sdr)
print(np.argwhere(multi_hot[0,2]>0))

[30, 95, 223, 238, 27, 203, 429, 225, 459, 284, 44]
[[ 27]
 [ 30]
 [ 38]
 [ 95]
 [203]
 [223]
 [225]
 [238]
 [284]
 [429]
 [459]]


In [55]:
def get_batches(arr, batch_size, seq_length):
    '''Create a generator that returns batches of size
       batch_size x seq_length from arr
       
       Arguments
       ---------
       arr: Array you want to make batches from
       batch_size: Batch size, the number of sequences per batch
       seq_length: Number of encoded chars in a sequence
    '''
    
    batch_size_total = batch_size * seq_length
    n_batches = len(arr) // batch_size_total
    
    arr = arr[:n_batches * batch_size_total]
    arr = arr.reshape((batch_size, -1))
    
    for n in range(0, arr.shape[1], seq_length):
        x = arr[:, n:n+seq_length]
        y = np.zeros_like(x) 
        try:
            y[:, :-1], y[:, -1] = x[:,1:], arr[:, n+seq_length]
        except IndexError:
            y[:, :-1], y[:, -1] = x[:,1:], arr[:,0] 
        yield x, y 
        

In [56]:
batches = get_batches(encoded, 8, 50)
x, y = next(batches)

In [57]:
# check if GPU is available
train_on_gpu = torch.cuda.is_available()
if(train_on_gpu):
    print('Training on GPU!')
else: 
    print('No GPU available, training on CPU; consider making n_epochs very small.')

Training on GPU!


In [58]:
class CharRNN(nn.Module):
    def __init__(self, tokens, n_hidden=612, n_layers=4, drop_prob=0.5, lr=0.001):
        super().__init__()
        self.drop_prob = drop_prob
        self.n_layers = n_layers
        self.n_hidden = n_hidden
        self.lr = lr
        
        self.chars = tokens
        self.int2char = dict(enumerate(self.chars))
        self.char2int = {ch:ii for ii, ch in self.int2char.items()}
        
        self.lstm = nn.LSTM(NumBits, n_hidden, n_layers, 
                            dropout=drop_prob, batch_first=True)
        self.dropout = nn.Dropout(drop_prob)
        
        self.fc = nn.Linear(n_hidden, NumBits)
        
    def forward(self, x, hidden):
        r_output, hidden = self.lstm(x,hidden)
        
        out = self.dropout(r_output)
        
        out = out.contiguous().view(-1, self.n_hidden)
        
        out = self.fc(out)
        
        return out, hidden
    
    def init_hidden(self, batch_size):
        weight = next(self.parameters()).data
        
        if (train_on_gpu):
            hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda(),
                  weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda())
        else:
            hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_(),
                      weight.new(self.n_layers, batch_size, self.n_hidden).zero_())
        
        return hidden

In [59]:
def train(net, data, epochs=10, batch_size=10, seq_length=50, lr=0.001, clip=5, val_frac=0.1, print_every=10):
    ''' Training a network 
    
        Arguments
        ---------
        
        net: CharRNN network
        data: text data to train the network
        epochs: Number of epochs to train
        batch_size: Number of mini-sequences per mini-batch, aka batch size
        seq_length: Number of character steps per mini-batch
        lr: learning rate
        clip: gradient clipping
        val_frac: Fraction of data to hold out for validation
        print_every: Number of steps for printing training and validation loss
    
    '''
    net.train()
    
    opt = torch.optim.Adam(net.parameters(), lr=lr)
    criterion = nn.BCEWithLogitsLoss()
    
    # create training and validation data
    val_idx = int(len(data)*(1-val_frac))
    data, val_data = data[:val_idx], data[val_idx:]
    
    if(train_on_gpu):
        net.cuda()
    
    counter = 0
    n_chars = NumBits
    for e in range(epochs):
        # initialize hidden state
        h = net.init_hidden(batch_size)
        
        for x, y in get_batches(data, batch_size, seq_length):
            counter += 1
            
            # One-hot encode our data and make them Torch tensors
            x = multi_hot_encoder(x, n_chars)
            y = multi_hot_encoder(y, n_chars)
            inputs, targets = torch.from_numpy(x), torch.from_numpy(y)
            
            if(train_on_gpu):
                inputs, targets = inputs.cuda(), targets.cuda()

            # Creating new variables for the hidden state, otherwise
            # we'd backprop through the entire training history
            h = tuple([each.data for each in h])

            # zero accumulated gradients
            net.zero_grad()
            
            # get the output from the model
            output, h = net(inputs, h)
            
            # calculate the loss and perform backprop
            loss = criterion(output, targets.view(batch_size*seq_length, NumBits))
            loss.backward()
            # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
            nn.utils.clip_grad_norm_(net.parameters(), clip)
            opt.step()
            
            # loss stats
            if counter % print_every == 0:
                # Get validation loss
                val_h = net.init_hidden(batch_size)
                val_losses = []
                net.eval()
                for x, y in get_batches(val_data, batch_size, seq_length):
                    # One-hot encode our data and make them Torch tensors
                    x = multi_hot_encoder(x, n_chars)
                    y = multi_hot_encoder(y, n_chars)
                    x, y = torch.from_numpy(x), torch.from_numpy(y)
                    
                    # Creating new variables for the hidden state, otherwise
                    # we'd backprop through the entire training history
                    val_h = tuple([each.data for each in val_h])
                    
                    inputs, targets = x, y
                    if(train_on_gpu):
                        inputs, targets = inputs.cuda(), targets.cuda()

                    output, val_h = net(inputs, val_h)
                    val_loss = criterion(output, targets.view(batch_size*seq_length, NumBits))
                
                    val_losses.append(val_loss.item())
                
                net.train() # reset to train mode after iterationg through validation data
                
                print("Epoch: {}/{}...".format(e+1, epochs),
                      "Step: {}...".format(counter),
                      "Loss: {:.4f}...".format(loss.item()),
                      "Val Loss: {:.4f}".format(np.mean(val_losses)))
                

In [60]:
# define and print the net
n_hidden=1024
n_layers=4

net = CharRNN(chars, n_hidden, n_layers)
print(net)

CharRNN(
  (lstm): LSTM(512, 1024, num_layers=4, batch_first=True, dropout=0.5)
  (dropout): Dropout(p=0.5, inplace=False)
  (fc): Linear(in_features=1024, out_features=512, bias=True)
)


In [ ]:
batch_size = 64
seq_length = 160 #max length verses
n_epochs = 5000 # start smaller if you are just testing initial behavior

# train the model
train(net, encoded, epochs=n_epochs, batch_size=batch_size, seq_length=seq_length, lr=0.001, print_every=10)

Epoch: 1/5000... Step: 10... Loss: 0.0878... Val Loss: 0.0872
Epoch: 1/5000... Step: 20... Loss: 0.0864... Val Loss: 0.0851
Epoch: 1/5000... Step: 30... Loss: 0.0850... Val Loss: 0.0832
Epoch: 1/5000... Step: 40... Loss: 0.0833... Val Loss: 0.0819
Epoch: 1/5000... Step: 50... Loss: 0.0822... Val Loss: 0.0805
Epoch: 1/5000... Step: 60... Loss: 0.0813... Val Loss: 0.0796
Epoch: 2/5000... Step: 70... Loss: 0.0806... Val Loss: 0.0791
Epoch: 2/5000... Step: 80... Loss: 0.0801... Val Loss: 0.0789
Epoch: 2/5000... Step: 90... Loss: 0.0803... Val Loss: 0.0789
Epoch: 2/5000... Step: 100... Loss: 0.0801... Val Loss: 0.0789
Epoch: 2/5000... Step: 110... Loss: 0.0801... Val Loss: 0.0789
Epoch: 2/5000... Step: 120... Loss: 0.0799... Val Loss: 0.0788
Epoch: 3/5000... Step: 130... Loss: 0.0799... Val Loss: 0.0788
Epoch: 3/5000... Step: 140... Loss: 0.0794... Val Loss: 0.0788
Epoch: 3/5000... Step: 150... Loss: 0.0797... Val Loss: 0.0788
Epoch: 3/5000... Step: 160... Loss: 0.0797... Val Loss: 0.0788
E

Epoch: 26/5000... Step: 1560... Loss: 0.0784... Val Loss: 0.0786
Epoch: 26/5000... Step: 1570... Loss: 0.0782... Val Loss: 0.0787
Epoch: 26/5000... Step: 1580... Loss: 0.0784... Val Loss: 0.0787
Epoch: 27/5000... Step: 1590... Loss: 0.0789... Val Loss: 0.0786
Epoch: 27/5000... Step: 1600... Loss: 0.0784... Val Loss: 0.0786
Epoch: 27/5000... Step: 1610... Loss: 0.0784... Val Loss: 0.0786
Epoch: 27/5000... Step: 1620... Loss: 0.0783... Val Loss: 0.0787
Epoch: 27/5000... Step: 1630... Loss: 0.0784... Val Loss: 0.0787
Epoch: 27/5000... Step: 1640... Loss: 0.0784... Val Loss: 0.0787
Epoch: 28/5000... Step: 1650... Loss: 0.0786... Val Loss: 0.0786
Epoch: 28/5000... Step: 1660... Loss: 0.0784... Val Loss: 0.0786
Epoch: 28/5000... Step: 1670... Loss: 0.0784... Val Loss: 0.0786
Epoch: 28/5000... Step: 1680... Loss: 0.0782... Val Loss: 0.0786
Epoch: 28/5000... Step: 1690... Loss: 0.0785... Val Loss: 0.0787
Epoch: 28/5000... Step: 1700... Loss: 0.0785... Val Loss: 0.0787
Epoch: 29/5000... Step: 1

Epoch: 47/5000... Step: 2830... Loss: 0.0785... Val Loss: 0.0786
Epoch: 47/5000... Step: 2840... Loss: 0.0783... Val Loss: 0.0787
Epoch: 47/5000... Step: 2850... Loss: 0.0784... Val Loss: 0.0786
Epoch: 47/5000... Step: 2860... Loss: 0.0785... Val Loss: 0.0787
Epoch: 48/5000... Step: 2870... Loss: 0.0785... Val Loss: 0.0786
Epoch: 48/5000... Step: 2880... Loss: 0.0784... Val Loss: 0.0786
Epoch: 48/5000... Step: 2890... Loss: 0.0785... Val Loss: 0.0786
Epoch: 48/5000... Step: 2900... Loss: 0.0783... Val Loss: 0.0787
Epoch: 48/5000... Step: 2910... Loss: 0.0785... Val Loss: 0.0787
Epoch: 48/5000... Step: 2920... Loss: 0.0786... Val Loss: 0.0787
Epoch: 49/5000... Step: 2930... Loss: 0.0785... Val Loss: 0.0787
Epoch: 49/5000... Step: 2940... Loss: 0.0784... Val Loss: 0.0787
Epoch: 49/5000... Step: 2950... Loss: 0.0784... Val Loss: 0.0786
Epoch: 49/5000... Step: 2960... Loss: 0.0785... Val Loss: 0.0786
Epoch: 49/5000... Step: 2970... Loss: 0.0783... Val Loss: 0.0787
Epoch: 49/5000... Step: 2

Epoch: 68/5000... Step: 4100... Loss: 0.0785... Val Loss: 0.0786
Epoch: 68/5000... Step: 4110... Loss: 0.0785... Val Loss: 0.0787
Epoch: 68/5000... Step: 4120... Loss: 0.0782... Val Loss: 0.0786
Epoch: 68/5000... Step: 4130... Loss: 0.0785... Val Loss: 0.0787
Epoch: 68/5000... Step: 4140... Loss: 0.0786... Val Loss: 0.0787
Epoch: 69/5000... Step: 4150... Loss: 0.0784... Val Loss: 0.0787
Epoch: 69/5000... Step: 4160... Loss: 0.0784... Val Loss: 0.0786
Epoch: 69/5000... Step: 4170... Loss: 0.0783... Val Loss: 0.0786
Epoch: 69/5000... Step: 4180... Loss: 0.0784... Val Loss: 0.0786
Epoch: 69/5000... Step: 4190... Loss: 0.0783... Val Loss: 0.0788
Epoch: 69/5000... Step: 4200... Loss: 0.0784... Val Loss: 0.0786
Epoch: 70/5000... Step: 4210... Loss: 0.0783... Val Loss: 0.0786
Epoch: 70/5000... Step: 4220... Loss: 0.0788... Val Loss: 0.0787
Epoch: 70/5000... Step: 4230... Loss: 0.0785... Val Loss: 0.0786
Epoch: 70/5000... Step: 4240... Loss: 0.0782... Val Loss: 0.0786
Epoch: 70/5000... Step: 4

Epoch: 89/5000... Step: 5370... Loss: 0.0786... Val Loss: 0.0787
Epoch: 89/5000... Step: 5380... Loss: 0.0783... Val Loss: 0.0786
Epoch: 89/5000... Step: 5390... Loss: 0.0783... Val Loss: 0.0787
Epoch: 89/5000... Step: 5400... Loss: 0.0785... Val Loss: 0.0786
Epoch: 89/5000... Step: 5410... Loss: 0.0783... Val Loss: 0.0787
Epoch: 89/5000... Step: 5420... Loss: 0.0785... Val Loss: 0.0786
Epoch: 90/5000... Step: 5430... Loss: 0.0785... Val Loss: 0.0787
Epoch: 90/5000... Step: 5440... Loss: 0.0788... Val Loss: 0.0786
Epoch: 90/5000... Step: 5450... Loss: 0.0785... Val Loss: 0.0787
Epoch: 90/5000... Step: 5460... Loss: 0.0783... Val Loss: 0.0787
Epoch: 90/5000... Step: 5470... Loss: 0.0781... Val Loss: 0.0787
Epoch: 90/5000... Step: 5480... Loss: 0.0786... Val Loss: 0.0786
Epoch: 90/5000... Step: 5490... Loss: 0.0785... Val Loss: 0.0786
Epoch: 91/5000... Step: 5500... Loss: 0.0785... Val Loss: 0.0786
Epoch: 91/5000... Step: 5510... Loss: 0.0785... Val Loss: 0.0787
Epoch: 91/5000... Step: 5

...

Epoch: 1377/5000... Step: 83960... Loss: 0.0421... Val Loss: 0.0516
Epoch: 1377/5000... Step: 83970... Loss: 0.0420... Val Loss: 0.0520
Epoch: 1377/5000... Step: 83980... Loss: 0.0417... Val Loss: 0.0519
Epoch: 1377/5000... Step: 83990... Loss: 0.0422... Val Loss: 0.0520
Epoch: 1378/5000... Step: 84000... Loss: 0.0424... Val Loss: 0.0520
Epoch: 1378/5000... Step: 84010... Loss: 0.0416... Val Loss: 0.0522
Epoch: 1378/5000... Step: 84020... Loss: 0.0418... Val Loss: 0.0519
Epoch: 1378/5000... Step: 84030... Loss: 0.0414... Val Loss: 0.0520
Epoch: 1378/5000... Step: 84040... Loss: 0.0415... Val Loss: 0.0522
Epoch: 1378/5000... Step: 84050... Loss: 0.0419... Val Loss: 0.0523
Epoch: 1379/5000... Step: 84060... Loss: 0.0421... Val Loss: 0.0523
Epoch: 1379/5000... Step: 84070... Loss: 0.0413... Val Loss: 0.0522
Epoch: 1379/5000... Step: 84080... Loss: 0.0418... Val Loss: 0.0522
Epoch: 1379/5000... Step: 84090... Loss: 0.0415... Val Loss: 0.0523
Epoch: 1379/5000... Step: 84100... Loss: 0.0415.

Epoch: 1397/5000... Step: 85170... Loss: 0.0392... Val Loss: 0.0538
Epoch: 1397/5000... Step: 85180... Loss: 0.0394... Val Loss: 0.0536
Epoch: 1397/5000... Step: 85190... Loss: 0.0393... Val Loss: 0.0541
Epoch: 1397/5000... Step: 85200... Loss: 0.0388... Val Loss: 0.0544
Epoch: 1397/5000... Step: 85210... Loss: 0.0395... Val Loss: 0.0539
Epoch: 1398/5000... Step: 85220... Loss: 0.0397... Val Loss: 0.0538
Epoch: 1398/5000... Step: 85230... Loss: 0.0391... Val Loss: 0.0538
Epoch: 1398/5000... Step: 85240... Loss: 0.0394... Val Loss: 0.0537
Epoch: 1398/5000... Step: 85250... Loss: 0.0393... Val Loss: 0.0541
Epoch: 1398/5000... Step: 85260... Loss: 0.0392... Val Loss: 0.0538
Epoch: 1398/5000... Step: 85270... Loss: 0.0396... Val Loss: 0.0539
Epoch: 1399/5000... Step: 85280... Loss: 0.0395... Val Loss: 0.0540
Epoch: 1399/5000... Step: 85290... Loss: 0.0387... Val Loss: 0.0538
Epoch: 1399/5000... Step: 85300... Loss: 0.0391... Val Loss: 0.0541
Epoch: 1399/5000... Step: 85310... Loss: 0.0391.

Epoch: 1417/5000... Step: 86380... Loss: 0.0382... Val Loss: 0.0549
Epoch: 1417/5000... Step: 86390... Loss: 0.0375... Val Loss: 0.0549
Epoch: 1417/5000... Step: 86400... Loss: 0.0376... Val Loss: 0.0550
Epoch: 1417/5000... Step: 86410... Loss: 0.0375... Val Loss: 0.0550
Epoch: 1417/5000... Step: 86420... Loss: 0.0370... Val Loss: 0.0552
Epoch: 1417/5000... Step: 86430... Loss: 0.0374... Val Loss: 0.0549
Epoch: 1418/5000... Step: 86440... Loss: 0.0379... Val Loss: 0.0550
Epoch: 1418/5000... Step: 86450... Loss: 0.0375... Val Loss: 0.0553
Epoch: 1418/5000... Step: 86460... Loss: 0.0374... Val Loss: 0.0552
Epoch: 1418/5000... Step: 86470... Loss: 0.0374... Val Loss: 0.0550
Epoch: 1418/5000... Step: 86480... Loss: 0.0370... Val Loss: 0.0553
Epoch: 1418/5000... Step: 86490... Loss: 0.0378... Val Loss: 0.0551
Epoch: 1419/5000... Step: 86500... Loss: 0.0374... Val Loss: 0.0552
Epoch: 1419/5000... Step: 86510... Loss: 0.0372... Val Loss: 0.0553
Epoch: 1419/5000... Step: 86520... Loss: 0.0373.

Epoch: 1436/5000... Step: 87590... Loss: 0.0358... Val Loss: 0.0563
Epoch: 1437/5000... Step: 87600... Loss: 0.0364... Val Loss: 0.0565
Epoch: 1437/5000... Step: 87610... Loss: 0.0356... Val Loss: 0.0564
Epoch: 1437/5000... Step: 87620... Loss: 0.0358... Val Loss: 0.0564
Epoch: 1437/5000... Step: 87630... Loss: 0.0362... Val Loss: 0.0561
Epoch: 1437/5000... Step: 87640... Loss: 0.0357... Val Loss: 0.0562
Epoch: 1437/5000... Step: 87650... Loss: 0.0361... Val Loss: 0.0563
Epoch: 1438/5000... Step: 87660... Loss: 0.0366... Val Loss: 0.0565
Epoch: 1438/5000... Step: 87670... Loss: 0.0358... Val Loss: 0.0564
Epoch: 1438/5000... Step: 87680... Loss: 0.0360... Val Loss: 0.0561
Epoch: 1438/5000... Step: 87690... Loss: 0.0360... Val Loss: 0.0561
Epoch: 1438/5000... Step: 87700... Loss: 0.0357... Val Loss: 0.0563
Epoch: 1438/5000... Step: 87710... Loss: 0.0361... Val Loss: 0.0561
Epoch: 1439/5000... Step: 87720... Loss: 0.0360... Val Loss: 0.0564
Epoch: 1439/5000... Step: 87730... Loss: 0.0359.

Epoch: 1456/5000... Step: 88800... Loss: 0.0343... Val Loss: 0.0569
Epoch: 1456/5000... Step: 88810... Loss: 0.0340... Val Loss: 0.0568
Epoch: 1457/5000... Step: 88820... Loss: 0.0348... Val Loss: 0.0573
Epoch: 1457/5000... Step: 88830... Loss: 0.0342... Val Loss: 0.0567
Epoch: 1457/5000... Step: 88840... Loss: 0.0342... Val Loss: 0.0570
Epoch: 1457/5000... Step: 88850... Loss: 0.0342... Val Loss: 0.0571
Epoch: 1457/5000... Step: 88860... Loss: 0.0338... Val Loss: 0.0568
Epoch: 1457/5000... Step: 88870... Loss: 0.0341... Val Loss: 0.0570
Epoch: 1458/5000... Step: 88880... Loss: 0.0344... Val Loss: 0.0574
Epoch: 1458/5000... Step: 88890... Loss: 0.0341... Val Loss: 0.0567
Epoch: 1458/5000... Step: 88900... Loss: 0.0341... Val Loss: 0.0567
Epoch: 1458/5000... Step: 88910... Loss: 0.0340... Val Loss: 0.0570
Epoch: 1458/5000... Step: 88920... Loss: 0.0339... Val Loss: 0.0569
Epoch: 1458/5000... Step: 88930... Loss: 0.0343... Val Loss: 0.0571
Epoch: 1459/5000... Step: 88940... Loss: 0.0342.

In [ ]:
model_dante = 'rnn_20_epoch.net'

checkpoint = {'n_hidden': net.n_hidden,
              'n_layers': net.n_layers,
              'state_dict': net.state_dict(),
              'tokens': net.chars}

with open(model_dante, 'wb') as f:
    torch.save(checkpoint, f)

In [ ]:
def predict(net, char, h=None, top_k=None):
        ''' Given a character, predict the next character.
            Returns the predicted character and the hidden state.
        '''
        
        # tensor inputs
        x = np.array([[net.char2int[char]]])
        x = one_hot_encoder(x, len(net.chars))
        inputs = torch.from_numpy(x)
        
        if(train_on_gpu):
            inputs = inputs.cuda()
        
        # detach hidden state from history
        h = tuple([each.data for each in h])
        # get the output of the model
        out, h = net(inputs, h)

        # get the character probabilities
        # apply softmax to get p probabilities for the likely next character giving x
        p = F.softmax(out, dim=1).data
        if(train_on_gpu):
            p = p.cpu() # move to cpu
        
        # get top characters
        # considering the k most probable characters with topk method
        if top_k is None:
            top_ch = np.arange(len(net.chars))
        else:
            p, top_ch = p.topk(top_k)
            top_ch = top_ch.numpy().squeeze()
        
        # select the likely next character with some element of randomness
        p = p.numpy().squeeze()
        char = np.random.choice(top_ch, p=p/p.sum())
        
        # return the encoded value of the predicted char and the hidden state
        return net.int2char[char], h

In [ ]:
def sample(net, size, prime='Il', top_k=None):
        
    if(train_on_gpu):
        net.cuda()
    else:
        net.cpu()
    
    net.eval() # eval mode
    
    # First off, run through the prime characters
    chars = [ch for ch in prime]
    h = net.init_hidden(1)
    for ch in prime:
        char, h = predict(net, ch, h, top_k=top_k)

    chars.append(char)
    
    # Now pass in the previous character and get a new one
    for ii in range(size):
        char, h = predict(net, chars[-1], h, top_k=top_k)
        chars.append(char)

    return ''.join(chars)

In [ ]:
print(sample(net, 1000, prime='This ', top_k=5))


In [ ]:
y

In [ ]:
x, y = next(batches)
print(x,y)